<a href="https://colab.research.google.com/github/sahilfatima/Basketball_Players/blob/main/Basketball_Player.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'social-power-nba:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1790%2F3098%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240430%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240430T145551Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D29d654fe54289282d7eba9c08d3b77c73440d7e0904c83da2da89aeaa3961f4278736f26c1220b319b05a1a33003c343160ab38de333455dbd6069a8b228cbd4bcf2a67aeec3fd5dd1425b5bfdd3d942dca4f5c4577d0a773215660fdcf3d45989c1447464f827d21b80a12ac3877376422d5bf8ccf1905a7c7a1e5cd16918fd5ee05a569afe701e8da19baace4c57fe05e24de3b08b997d7d4c8523bd1ac80ec7ed370320683bc2199607f8284aa07ff52bd35b27b1b2d5a0301698a2801ece029f343ebe25f7b67280304703e07a12c84f1196ddfb99a145751ce2bbfa4491d817e9530d720dcf1f8b8e27bf7ad724894f3f113daf0c9fd1274e80e41fa340,nba-players-stats:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1358%2F30676%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240430%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240430T145551Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D46ffaff0258189e9891c21f3d817a61919868bb885556b3f1d19189bbdd4db90a4d2375647df1b2849c7d56ddda2343577fed735133b41f9f7a372c4775d55046722abba1dea3806d7ff867b09bb6d688993eb47a5f95055887ae5f5493e2372f54987e74c4702b2a0d4ddee1c23a7dd6eb74d71410796dfa56ffd5ac3d0a3ebc0418416fb494c6c3ecfbd6e080edf72416fe35e6dabb42a20d05fbc635efa8616d2985ae2dfddf533b30121a09b44eab7db5f4f00cf0cc289bd4bdbce315377c107cfeb9d8f803a3028b4d75f4246ab2add8ae75e1a103e1f6aa845619f68c694895db23b37d5085aa58a7264754a364db14d5027164325c1665208b179bb33,basketball-players-stats-per-season-49-leagues:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F832499%2F1675893%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240430%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240430T145551Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D462bfb9b4ae5e5cb1bed54fe5f231f38e92df460dced7f58142336094550ebbe33cc0521cb9b9604b367bab0b1dc5b2c76d245f1e3a2a2a87e438649a6ba9ce1abc70b875a6488ec6474004920e25b9f50d386374e96e10fb6bfc30db8aa8ebbf02e0f1c166b5b2772caf25c06ac7e248e08c0b456de81c80eff698b106965bbfcd1c5245806bc598857ed49042edad63ccb571318282a37cc6ec02cf27129c4af6b81a0a8944fd1045f2df5276df03337ed028f5e6c36fbad98fe5976f646c0e1fdf650d2407a1b8bc06e1bfc7211fe2cf7a755f215a7a99d377cf60ed8e3720b4cb01f3a9ea9c490eecceb4d4c2e910d808ad24da937baf4c7993f10133f56,nba-players-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F98271%2F6687390%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240430%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240430T145551Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4d9cd8556eaa03c5a678063a0665b111f4373d8d7be591cace4798c0ad0b2cb5258f81dbb356660884f9b610155021c513b2b59299d6930fc5f62c837a64d1211df9968f23d5aec4241136e556adda3c7f2ee81c382032ee86c22570c69aac5722a7e0c203349ab75a3dd290386522359ec66e3f0f017a73f6d0e08b17466b82677d1e4993d4b018eb76528ed8aaac051e00fb7e052fc9f328e35986f6edf373c8a8c0ea7a0ee80cf9df8c0364416039cb92e3ee26fd285077f45ebaab9705a138fb5d19507452d3a1eabcb8337f460efb94d90b037abe1abbdc10f81f289e799339798ca0373d6dae9ecad7114fa036d2f8b1b184662f536f49d99f76b9520f'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
#Importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [ ]:
#Loading dataset
dataset = pd.read_csv("../input/social-power-nba/nba_2017_nba_players_with_salary.csv")
dataset1 = pd.read_csv("../input/nba-players-data/all_seasons.csv")

In [ ]:
dataset.head(5)

In [ ]:
dataset1 = dataset1.loc[(dataset1['season'] == "2016-17")]

In [ ]:
dataset1.head(5)

In [ ]:
players_height_weight = dataset1[['player_name','player_height' ,'player_weight']].copy()

In [ ]:
players_height_weight

In [ ]:
players_height_weight.rename(columns={'player_name': 'PLAYER'}, inplace=True)
players_height_weight.rename(columns={'player_height': 'HEIGHT'}, inplace=True)
players_height_weight.rename(columns={'player_weight': 'WEIGHT'}, inplace=True)

In [ ]:
players_height_weight

In [ ]:
df = dataset.merge(players_height_weight, how="inner", on="PLAYER")

In [ ]:
df

In [ ]:
df.columns

In [ ]:
del df['3P%']
del df['2P%']
del df['eFG%']
del df['FT%']
del df['FG%']
del df['ORB']
del df['DRB']
del df['PACE']
del df['AGE']
del df['WEIGHT']

In [ ]:
del df['Unnamed: 0']
del df['Rk']
del df['TEAM']

In [ ]:
df

In [ ]:
position_df = df['POSITION']

In [ ]:
df.describe().T

In [ ]:
x = df.iloc[:,2:].values
y = df.iloc[:,0]

In [ ]:
x

In [ ]:
y

In [ ]:
from sklearn import preprocessing

In [ ]:
corr = df.corr()

In [ ]:
corr

In [ ]:
plt.subplots(figsize=(20,20))
ax = plt.axes()
ax.set_title("NBA Players Correlation Heatmap:  2016-2017 Season ")
sns.heatmap(corr, xticklabels = corr.columns.values , yticklabels = corr.columns.values)

In [ ]:
figx = px.scatter_matrix(df,dimensions=["SALARY_MILLIONS","WINS_RPM","PIE","HEIGHT" ,"BLK" , "POINTS","TRB"], color="POSITION")

In [ ]:
figx

In [ ]:
sns.lmplot(x="SALARY_MILLIONS", y="WINS_RPM", data=df)

In [ ]:
sns.lmplot(x="SALARY_MILLIONS", y="PIE", data=df)

In [ ]:
sns.lmplot(x="HEIGHT", y="TRB", data=df)

In [ ]:
sns.lmplot(x="HEIGHT", y="BLK", data=df)

In [ ]:
datasetcr = preprocessing.scale(x)

In [ ]:
datasetcr

In [ ]:
eig_vals , eig_vecs = np.linalg.eig(corr)

In [ ]:
eig_vals

In [ ]:
eig_vecs

In [ ]:
from sklearn.decomposition import PCA


In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=4)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2' ,'principal component 3' ,'principal component 4' ])

In [ ]:
pca.explained_variance_ratio_

In [ ]:
print(pca.explained_variance_ratio_ * 100)

In [ ]:
principalDf

In [ ]:
finalDf = pd.concat([principalDf, y ,position_df ], axis = 1)

In [ ]:
finalDf

In [ ]:
pca.fit(x)

In [ ]:
pcs = pca.components_


In [ ]:
pcs

In [ ]:
columns=dataset.columns.values

In [ ]:
columns

In [ ]:
x

In [ ]:
finalDf

In [ ]:
pc1 = finalDf.iloc[:,0]

In [ ]:
pc1

In [ ]:
pc2 = finalDf.iloc[:,1]

In [ ]:
pc2

In [ ]:
pc3 = finalDf.iloc[:,2]

In [ ]:
pc3

In [ ]:
fig = px.scatter(finalDf, x="principal component 1" , y="principal component 2"  ,hover_name ="PLAYER",  size_max=100 , color="POSITION"   )

In [ ]:
fig

In [ ]:
fig1 = px.scatter(finalDf, x="principal component 1" , y="principal component 3",hover_name ="PLAYER",  size_max=100 , color="POSITION"   )

In [ ]:
fig1

**Observation : les joueurs sont placés d'une facon particulers :**
* Les PG-SG : Au dessus de l'axe des abscies
* LES C-PF : Au dessous de l'axe des abscies

In [ ]:
fig2 = px.scatter(finalDf, x="principal component 2" , y="principal component 3",hover_name="PLAYER", size_max=100 , color="POSITION"  )

In [ ]:
fig2

In [ ]:
fig3 = px.scatter(finalDf, x="principal component 1" , y="principal component 4",hover_name="PLAYER", size_max=100 , color="POSITION"  )

In [ ]:
fig3

In [ ]:
fig4 = px.scatter(finalDf, x="principal component 3" , y="principal component 4",hover_name="PLAYER", size_max=100 , color="POSITION"  )

In [ ]:
fig4

In [ ]:
df.loc[(df['PLAYER'] == "Rudy Gay")]

In [ ]:
df.loc[(df['PLAYER'] == "JaVale McGee")]

In [ ]:
print(df.loc[(df['PLAYER'] == "Isaiah Thomas")] )

In [ ]:
print(df.loc[(df['PLAYER'] == "Kristaps Porzingis")] )